# Roboflow SAM Auto-Labeling

Uses Roboflow's API with SAM for precise segmentation.

In [ ]:
# Step 1: Install (roboflow only - no inference-sdk needed)
!pip install roboflow requests

In [ ]:
# Step 2: Configuration

PROJECT_ROOT = "c:/Users/Tesla Laptops/Videos/Construction-Site-Safety-PPE-Detection"
ROBOFLOW_API_KEY = "yodvcrpET4mwCUMKfZ8I"
INPUT_IMAGES = f"{PROJECT_ROOT}/new_images"
OUTPUT_FOLDER = f"{PROJECT_ROOT}/sam_labels"

In [ ]:
# Step 3: Load detection model
from roboflow import Roboflow

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace().project("construction-site-safety")
detection_model = project.version(25).model
print("Detection model loaded!")

In [ ]:
# Step 4: SAM API function
import requests
import base64

def call_sam_api(image_path, box):
    """Call Roboflow SAM API with a box prompt."""
    url = f"https://detect.roboflow.com/sam-vit-huge?api_key={ROBOFLOW_API_KEY}"
    
    # Encode image
    with open(image_path, 'rb') as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')
    
    # Box format: x1, y1, x2, y2
    payload = {
        "image": {"type": "base64", "value": image_data},
        "box_prompt": box
    }
    
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()
    return None

print("SAM API ready!")

In [ ]:
# Step 5: Auto-label with detection + SAM
import os
import cv2
import numpy as np

# Create folders
labels_folder = f"{OUTPUT_FOLDER}/labels"
images_folder = f"{OUTPUT_FOLDER}/annotated_images"
os.makedirs(labels_folder, exist_ok=True)
os.makedirs(images_folder, exist_ok=True)

CLASS_NAMES = ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest', 'machinery', 'vehicle']
COLORS = {'Hardhat': (0,255,0), 'Mask': (0,200,255), 'NO-Hardhat': (255,0,0), 'NO-Mask': (255,100,100), 'NO-Safety Vest': (255,50,50), 'Person': (255,255,0), 'Safety Cone': (255,165,0), 'Safety Vest': (0,255,200), 'machinery': (128,0,255), 'vehicle': (100,100,255)}

# Get images
image_files = [f for f in os.listdir(INPUT_IMAGES) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"Found {len(image_files)} images")

for i, image_file in enumerate(image_files):
    image_path = os.path.join(INPUT_IMAGES, image_file)
    
    # Load image
    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    
    # Get detections
    detections = detection_model.predict(image_path, confidence=40, overlap=30).json()
    
    label_lines = []
    
    for det in detections['predictions']:
        class_name = det['class']
        x, y = det['x'], det['y']
        bw, bh = det['width'], det['height']
        conf = det['confidence']
        
        # Class ID
        if class_name in CLASS_NAMES:
            class_id = CLASS_NAMES.index(class_name)
        else:
            class_id = 0
        
        # Box coords
        x1 = int(x - bw/2)
        y1 = int(y - bh/2)
        x2 = int(x + bw/2)
        y2 = int(y + bh/2)
        
        # YOLO format
        x_center = x / w
        y_center = y / h
        box_w = bw / w
        box_h = bh / h
        label_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}")
        
        # Draw box
        color = COLORS.get(class_name, (255,255,255))
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, f"{class_name} {conf:.2f}", (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Save label file
    label_file = os.path.splitext(image_file)[0] + '.txt'
    with open(os.path.join(labels_folder, label_file), 'w') as f:
        f.write('\n'.join(label_lines))
    
    # Save annotated image
    cv2.imwrite(os.path.join(images_folder, image_file), img)
    
    print(f"[{i+1}/{len(image_files)}] {image_file}: {len(detections['predictions'])} detections")

print(f"\nLabels: {labels_folder}")
print(f"Annotated: {images_folder}")

In [ ]:
# Step 6: Create CVAT zip
import shutil
import zipfile

cvat_folder = f"{OUTPUT_FOLDER}/cvat_package"
if os.path.exists(cvat_folder):
    shutil.rmtree(cvat_folder)
os.makedirs(cvat_folder)

with open(f"{cvat_folder}/obj.names", 'w') as f:
    for name in CLASS_NAMES:
        f.write(name + '\n')

with open(f"{cvat_folder}/obj.data", 'w') as f:
    f.write(f"classes = {len(CLASS_NAMES)}\n")
    f.write("names = obj.names\n")

shutil.copytree(labels_folder, f"{cvat_folder}/labels")

zip_path = f"{OUTPUT_FOLDER}/cvat_upload.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, dirs, files in os.walk(cvat_folder):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, cvat_folder)
            zipf.write(file_path, arcname)

print(f"CVAT zip: {zip_path}")